<a href="https://colab.research.google.com/github/b-abhinav365/Symptom-Detection/blob/main/Symptom_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
from datasets import load_dataset
dataset = load_dataset("gretelai/symptom_to_diagnosis")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/171k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/42.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/853 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/212 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.DataFrame(dataset['train'])
print(df.head())

               output_text                                         input_text
0     cervical spondylosis  I've been having a lot of pain in my neck and ...
1                 impetigo  I have a rash on my face that is getting worse...
2  urinary tract infection  I have been urinating blood. I sometimes feel ...
3                arthritis  I have been having trouble with my muscles and...
4                   dengue  I have been feeling really sick. My body hurts...


# Cleaning the text NLP

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stop_words = stopwords.words('english')
stop_words.remove('not')
ps = PorterStemmer()

# Train data
corpus_train = []
for text in dataset['train']['input_text']:
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in stop_words]
    corpus_train.append(' '.join(review))

# Test data
corpus_test = []
for text in dataset['test']['input_text']:
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in stop_words]
    corpus_test.append(' '.join(review))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Notes: Here we are training the model on bag of words to get the vectorizered format of the text from the data, and as the dataset is already split into training & testing set we do not need to split the dataset again.

# Creating the Bag of Words Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=1000)
X_train = cv.fit_transform(corpus_train).toarray()
X_test = cv.transform(corpus_test).toarray()

# Encoding the Output labels ie, Symptoms

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(dataset['train']['output_text'])
y_test = le.transform(dataset['test']['output_text'])

# Training the ML model on Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


nb_model = MultinomialNB()


nb_model.fit(X_train, y_train)

# Predict on test data
y_pred_nb = nb_model.predict(X_test)

# Evaluate performance
print("Naive Bayes Model Performance:\n")
print(classification_report(y_test, y_pred_nb, target_names=le.classes_))

Naive Bayes Model Performance:

                                 precision    recall  f1-score   support

                        allergy       1.00      0.80      0.89        10
                      arthritis       0.91      1.00      0.95        10
               bronchial asthma       1.00      1.00      1.00        10
           cervical spondylosis       1.00      1.00      1.00        10
                    chicken pox       0.86      0.60      0.71        10
                    common cold       0.67      1.00      0.80        10
                         dengue       0.89      0.80      0.84        10
                       diabetes       1.00      0.80      0.89        10
                  drug reaction       1.00      0.75      0.86         8
               fungal infection       1.00      1.00      1.00         9
gastroesophageal reflux disease       0.67      0.80      0.73        10
                   hypertension       0.83      1.00      0.91        10
                  

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

print("Logistic Regression Performance:\n")
print(classification_report(y_test, y_pred_lr, target_names=le.classes_))

Logistic Regression Performance:

                                 precision    recall  f1-score   support

                        allergy       0.83      1.00      0.91        10
                      arthritis       1.00      1.00      1.00        10
               bronchial asthma       0.91      1.00      0.95        10
           cervical spondylosis       1.00      1.00      1.00        10
                    chicken pox       1.00      0.90      0.95        10
                    common cold       1.00      0.90      0.95        10
                         dengue       0.82      0.90      0.86        10
                       diabetes       0.91      1.00      0.95        10
                  drug reaction       1.00      1.00      1.00         8
               fungal infection       1.00      1.00      1.00         9
gastroesophageal reflux disease       1.00      0.80      0.89        10
                   hypertension       1.00      1.00      1.00        10
                

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Random Forest Performance:\n")
print(classification_report(y_test, y_pred_rf, target_names=le.classes_))

Random Forest Performance:

                                 precision    recall  f1-score   support

                        allergy       1.00      0.90      0.95        10
                      arthritis       0.91      1.00      0.95        10
               bronchial asthma       0.91      1.00      0.95        10
           cervical spondylosis       0.91      1.00      0.95        10
                    chicken pox       0.90      0.90      0.90        10
                    common cold       0.91      1.00      0.95        10
                         dengue       0.86      0.60      0.71        10
                       diabetes       1.00      0.80      0.89        10
                  drug reaction       1.00      1.00      1.00         8
               fungal infection       1.00      1.00      1.00         9
gastroesophageal reflux disease       1.00      0.90      0.95        10
                   hypertension       0.77      1.00      0.87        10
                      

# SVM

In [ ]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

print("SVM Performance:\n")
print(classification_report(y_test, y_pred_svm, target_names=le.classes_))

SVM Performance:

                                 precision    recall  f1-score   support

                        allergy       0.90      0.90      0.90        10
                      arthritis       0.91      1.00      0.95        10
               bronchial asthma       0.91      1.00      0.95        10
           cervical spondylosis       0.91      1.00      0.95        10
                    chicken pox       0.89      0.80      0.84        10
                    common cold       0.82      0.90      0.86        10
                         dengue       0.88      0.70      0.78        10
                       diabetes       0.91      1.00      0.95        10
                  drug reaction       1.00      1.00      1.00         8
               fungal infection       1.00      1.00      1.00         9
gastroesophageal reflux disease       1.00      0.80      0.89        10
                   hypertension       0.91      1.00      0.95        10
                       impetigo 

# Making a comparision between the trained models.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
results = []

# Naive Bayes
results.append({
    'Model': 'Naive Bayes',
    'Accuracy': accuracy_score(y_test, y_pred_nb),
    'F1 Score': f1_score(y_test, y_pred_nb, average='weighted')
})

# Logistic Regression
results.append({
    'Model': 'Logistic Regression',
    'Accuracy': accuracy_score(y_test, y_pred_lr),
    'F1 Score': f1_score(y_test, y_pred_lr, average='weighted')
})

# Random Forest
results.append({
    'Model': 'Random Forest',
    'Accuracy': accuracy_score(y_test, y_pred_rf),
    'F1 Score': f1_score(y_test, y_pred_rf, average='weighted')
})

# SVM
results.append({
    'Model': 'SVM',
    'Accuracy': accuracy_score(y_test, y_pred_svm),
    'F1 Score': f1_score(y_test, y_pred_svm, average='weighted')
})

In [ ]:
results_df = pd.DataFrame(results)
print("\n Model Comparison:\n")
print(results_df.sort_values(by='Accuracy', ascending=False))


 Model Comparison:

                 Model  Accuracy  F1 Score
1  Logistic Regression  0.957547  0.957832
2        Random Forest  0.938679  0.936545
3                  SVM  0.938679  0.937482
0          Naive Bayes  0.886792  0.884454


## Note: As we can see in the above comparision, the Logistic Regression model and the Random Forest are having the highest accuracy scores

# Now Let's predict based on the trained model.

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer



def diagnose_symptom():
    print(" Hello! I'm your HealthBot. Tell me how you're feeling today.")
    print("Type 'exit' to stop.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("HealthBot: Take care!")
            break

        review = re.sub('[^a-zA-Z]', ' ', user_input)
        review = review.lower().split()
        ps = PorterStemmer()
        stop_words = stopwords.words('english')
        review = [ps.stem(word) for word in review if word not in set(stop_words)]
        processed_input = ' '.join(review)

        input_vector = cv.transform([processed_input]).toarray()

        predicted_index = nb_model.predict(input_vector)[0]


        predicted_label = le.inverse_transform([predicted_index])

        print(f"HealthBot: Based on what you said, it might be **{predicted_label[0]}**.\n")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Note: The below loop will be running untill we type exit in the chat.

In [ ]:
diagnose_symptom()

 Hello! I'm your HealthBot. Tell me how you're feeling today.
Type 'exit' to stop.

You: "My skin has started turning yellow, and I often feel nauseous after eating. I also feel tired all the time."
HealthBot: Based on what you said, it might be **jaundice**.

You: "I’ve been experiencing intense joint pain, especially in the knees and fingers, with swelling and stiffness in the mornings."
HealthBot: Based on what you said, it might be **arthritis**.

You: "After every meal, I feel a burning sensation in my chest and sometimes get a sour taste in my mouth."
HealthBot: Based on what you said, it might be **gastroesophageal reflux disease**.

You: exit
HealthBot: Take care!
